In [ ]:
# pip install flask flask-socketio

In [ ]:
index_html_string = '''  
<!DOCTYPE html>
<html>
<head>
  <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
  <style> 
    .active {
      stroke: #000;
      stroke-width: 2px;
  } 
  </style> 
  <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.0.1/socket.io.js" 
          integrity="sha512-q/dWJ3kcmjBLU4Qc47E4A9kTB4m3wuTY7vkFJDTZKjTs8jhyGQnaUrxa0Ytd0ssMZhbNua9hE+E7Qv1j+DyZwA==" 
          crossorigin="anonymous">
  </script> 
</head>
<body>
  <script src="https://d3js.org/d3.v4.min.js"></script>
  <svg width="1000" height="500" style="border: 1px solid black">
    <circle cx="250" cy="250" r="200" style="fill:none;stroke:#FF7F7F;stroke-width:16px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1;z-index: 1;"/>
    <circle cx="750" cy="250" r="200" style="fill:none;stroke:#FF7F7F;stroke-width:16px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1;z-index: 1;"/>
  </svg>  
  <script type="text/javascript" charset="utf-8">
  
  
  var socket = io.connect('http://{{host}}:5000');  
  // socket.on('response', function (count) {
  //     document.getElementById('counter').innerHTML = count;
  // }); 
  
  var svg = d3.select("svg"),
      width = +svg.attr("width"),
      height = +svg.attr("height"),
      radius = 32; 
 
  var circles = [{},{},
               // dcx: default_center_X_coordinate
                 {dcx: 250, dcy: 250, color: '#1f77b4', id: 0}, 
                 {dcx: 750, dcy: 250, color: '#aec7e8', id: 1}]; 
  
  svg.selectAll("circle")
    .data(circles)
    .enter().append("circle")
      .attr("cx", function(d) { return d.dcx; })
      .attr("cy", function(d) { return d.dcy; })
      .attr("dcx", function(d) { return d.dcx; })
      .attr("dcy", function(d) { return d.dcy; })
      .attr("r", radius)
      .attr("id", function(d) { return d.id; })
      .style("fill", function(d, i) { return d.color; })
      .call(d3.drag()
        .on("start", dragstarted)
        .on("drag", dragged)
        .on("end", dragended));
   
  function dragstarted(d) {
    d3.select(this)
      .raise()
      .classed("active", true);
  }
  
  function keepInCircle(dcx, dcy, x, y, r) {
    const dx = x - dcx, 
          dy = y - dcy, 
       ratio = r / Math.sqrt(dx * dx + dy * dy);
    return (ratio<1) ? { x:dcx+dx*ratio, y:dcy+dy*ratio, dx:dx, dy:dy } : {x:x, y:y, dx:dx, dy:dy}  
  } 
  
  function dragged(d) {   
    var obj = keepInCircle(d.dcx, d.dcy, d3.event.x, d3.event.y, 200);
    d3.select(this)
      .attr("cx", obj.x)
      .attr("cy", obj.y);
    socket.emit("joystick", {'id':d.id, 'x':obj.dx, 'y':obj.dy}); 
  }
  
  function dragended(d) {
    d3.select(this)
      .attr("cx", d.dcx)
      .attr("cy", d.dcy)
      .classed("active", false);
  }
  
  </script>
</body>
</html>
'''
from flask import Flask, render_template_string
from flask_socketio import SocketIO, send, emit
import queue

q_ctrl = queue.Queue()

app = Flask(__name__) 
socketio = SocketIO(app, logger=True, engineio_logger=True)

host = '127.0.0.1'
count = 0
@app.route("/")
def hello():
    global count
    return render_template_string(index_html_string, host=host)  

@socketio.on('joystick')
def joystick_receiver(js): 
    global q_ctrl 
    q_ctrl.put(js)
    
if __name__ == '__main__':
    socketio.run(app, host=host, port=5000) 